# Intersight Python SDK

If you are familiar Python, you can take advantage of the Intersight Python SDK available for install from the [Python Package Index](https://pypi.org/project/intersight/).

![Python SDK for Cisco Intersight](images/python_sdk_pip.png)

## Installation

Notice the `pip install intersight` command. That's the command used to install the Python SDK in your environment.  Be sure you are using Python >= 3.7, earlier versions of Python are not supported.

In [1]:
pip install intersight

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.



> Be sure to uninstall any conflicting versions of the SDK if you have previous installs.  You can check installed versions with pip list.

In [2]:
pip list | grep -i intersight

intersight          1.0.9.5808
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


> If you see Intersight-OpenAPI installed, you can run `pip unistall Intersight-OpenAPI`

## Using the Intersight Python SDK

To use the SDK, you must import required modules.  You can view help once you've imported any required modules.


In [3]:
import intersight

# View the help
help(intersight)


Help on package intersight:

NAME
    intersight - Cisco Intersight

DESCRIPTION
    Cisco Intersight is a management platform delivered as a service with embedded analytics for your Cisco and 3rd party IT infrastructure. This platform offers an intelligent level of management that enables IT organizations to analyze, simplify, and automate their environments in more advanced ways than the prior generations of tools. Cisco Intersight provides an integrated and intuitive management experience for resources in the traditional data center as well as at the edge. With flexible deployment options to address complex security needs, getting started with Intersight is quick and easy. Cisco Intersight has deep integration with Cisco UCS and HyperFlex systems allowing for remote deployment, configuration, and ongoing maintenance. The model-based deployment works for a single system in a remote location or hundreds of systems in a data center and enables rapid, standardized configuration and depl

### API Keys and User Authentication

Now let's use the Intersight Python SDK to connect to our Intersight environment.  First, you'll need an Intersight API Key ID and secret (private) key from your Intersight account.  From the Settings menu in Intersight, Select API Keys and Generate API Key.

![Generate API Key](images/generate_api_key.png)

A version 3 key can be used with the SDK and is recommended for long term compatibility with Intersight's API.

![Version 3 key](images/version_3_key.png)

The Generated API Key ID can be copied to the clipboard and used in API authentication below.  Also be sure to save the Secret (private) Key to a local file that only you have access to (a download option is provided by Intersight).

![Download Secret Key](images/secret_key_download.png)

### Intersight API Configuration

We'll create an API client and use it to connect with Intersight.

> Note that in DevNet labs the values below can be used as is.  For other environments, be sure to enter the values for your account in the `key_id` variable below as well as the `private_key_path`.  Also be sure that you are using v3 API keys from your Intersight account.  

In [4]:
import intersight

configuration = intersight.Configuration(
    signing_info=intersight.HttpSigningConfiguration(
        key_id='596cc79e5d91b400010d15ad/60ede8d07564612d3335edac/623354c07564612d33d9068d',
        private_key_path='./SecretKey.txt',
        signing_scheme=intersight.signing.SCHEME_HS2019,
        signed_headers=[intersight.signing.HEADER_HOST,
                        intersight.signing.HEADER_DATE,
                        intersight.signing.HEADER_DIGEST,
                        intersight.signing.HEADER_REQUEST_TARGET
                        ]
    )
)
api_client = intersight.ApiClient(configuration)


### Intersight Python SDK Query Examples

Now that you are authenticated, let's use the SDK to query server and related resources in Intersight.  You can read more on [Intersight's API query language](https://www.intersight.com/apidocs/introduction/query/), and to get an idea of the resources used by the API you can look at the URIs your browser uses (compute/physical-summaries below).

![Browser Compute Summary](images/browser_compute_phys_summary.png)

The Python example below sets up a query filter to get a count of servers and then uses the returned count to page through all servers in the account.  When getting server summaries, the select query parameter is used to only return certain server attributes.

In [5]:
import intersight.api.compute_api

# Create physical summary (server) class instance
api_instance = intersight.api.compute_api.ComputeApi(api_client)

# Find the count of servers in the account
server_query = api_instance.get_compute_physical_summary_list(count=True)
print(server_query)

# Intersight limits the number of items returned.  Page through returned results and select Name, Model, Serial
per_page = 50
query_select = "Name,Model,Serial"
for i in range(0, server_query.count, per_page):
    page_query = api_instance.get_compute_physical_summary_list(top=per_page, skip=i, select=query_select)
    print(page_query)


{'count': 10, 'object_type': 'mo.DocumentCount'}
{'object_type': 'compute.PhysicalSummary.List',
 'results': [{'class_id': 'compute.PhysicalSummary',
              'model': 'HXAF220C-M5SX',
              'moid': '5dee9c8e6176752d332e94c8',
              'name': 'C220-WZP2204097Z',
              'object_type': 'compute.PhysicalSummary',
              'serial': 'WZP2204097Z'},
             {'class_id': 'compute.PhysicalSummary',
              'model': 'HXAF220C-M5SX',
              'moid': '5dee9c8f6176752d332e94fa',
              'name': 'C220-WZP2204093R',
              'object_type': 'compute.PhysicalSummary',
              'serial': 'WZP2204093R'},
             {'class_id': 'compute.PhysicalSummary',
              'model': 'HXAF220C-M5SX',
              'moid': '5dee9c916176752d332e9583',
              'name': 'C220-WZP220400B3',
              'object_type': 'compute.PhysicalSummary',
              'serial': 'WZP220400B3'},
             {'class_id': 'compute.PhysicalSummary',
       

### Detailed Inventory

Building on the server inventory example above, we'll now look at Intersight's resource model to get information on server components like physical disks.  We can search in the API for specific disk Model numbers or search for Models that contain a certian substring.  Below we will search for physical disks Model numbers that contain a certain substring. 

In [6]:
import intersight.api.storage_api

# Create storage class instance
api_instance = intersight.api.storage_api.StorageApi(api_client)

# Setup query options
query_filter = "contains(Model,'UCS')"
query_select = "Dn,Model,RegisteredDevice,RunningFirmware"
query_expand = "RegisteredDevice($select=DeviceHostname),RunningFirmware($select=Version)"

# Get physcial disks that contain the Model substring defined above
storage_query = api_instance.get_storage_physical_disk_list(filter=query_filter, select=query_select, expand=query_expand)
print(storage_query)


{'object_type': 'storage.PhysicalDisk.List',
 'results': [{'class_id': 'storage.PhysicalDisk',
              'dn': '/redfish/v1/Systems/FCH2141JB8E/Storage/FMEZZ1-SAS/Drives/1',
              'model': 'UCS-HD300G10K12G',
              'moid': '5f35604f6176752d376223bf',
              'object_type': 'storage.PhysicalDisk',
              'registered_device': {'class_id': 'asset.DeviceRegistration',
                                    'device_hostname': ['C1-B2-UCSB-B200-M5'],
                                    'moid': '5f355faa6f72612d31b261c1',
                                    'object_type': 'asset.DeviceRegistration'},
              'running_firmware': [{'class_id': 'firmware.RunningFirmware',
                                    'moid': '5f35604f6176752d376223c5',
                                    'object_type': 'firmware.RunningFirmware',
                                    'version': '5705'}]},
             {'class_id': 'storage.PhysicalDisk',
              'dn': '/redfish/v1/S

## Configure Resources

### Create a new BIOS Policy

In [7]:
from intersight.api import bios_api
from intersight.model.bios_policy import BiosPolicy
from intersight.model.organization_organization_relationship import OrganizationOrganizationRelationship

organization = OrganizationOrganizationRelationship(moid="5deea1d16972652d33ba886b",
                                                    object_type="organization.Organization",
                                                    class_id="mo.MoRef",
                                                    )


bios_policy = BiosPolicy()
bios_policy.name = "CLive2022"
bios_policy.organization = organization
bios_policy.cpu_energy_performance = "performance"
bios_policy.cpu_performance = "hpc"

# Create a bios.Policy resource
bios_policy_instance = bios_api.BiosApi(api_client)
api_response = bios_policy_instance.create_bios_policy(bios_policy)


ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '289', 'Connection': 'keep-alive', 'Date': 'Thu, 07 Apr 2022 21:53:59 GMT', 'Set-Cookie': 'AWSALB=dePmLaeakyGnK68pKivYh7E4oeLYrX1Y2hktKAlnFRnbqzjFvvMJShLLFRd6E+yVHeZikRicjau/kJuHbfnbOICBrxwoh9BPWsJjgWUsQJDCfCVE3X0FH0CtBvE9; Expires=Thu, 14 Apr 2022 21:53:59 GMT; Path=/, AWSALBCORS=dePmLaeakyGnK68pKivYh7E4oeLYrX1Y2hktKAlnFRnbqzjFvvMJShLLFRd6E+yVHeZikRicjau/kJuHbfnbOICBrxwoh9BPWsJjgWUsQJDCfCVE3X0FH0CtBvE9; Expires=Thu, 14 Apr 2022 21:53:59 GMT; Path=/; SameSite=None; Secure', 'Server': 'nginx', 'X-Starship-TraceId': 'NB9335333b64a085ac15cd5cabc08135fe', 'Pragma': 'no-cache', 'Cache-Control': 'no-cache, no-store, must-revalidate', 'Content-Security-Policy': "base-uri 'self' https://cdn.intersight.com/; child-src 'self' https://www.intersight.com/ https://cdn.intersight.com/  https://cloudsso.cisco.com https://www.cisco.com/ https://id.cisco.co

### Update a BIOS Policy

### Delete a BIOS Policy

## Additional Examples

Several additional examples of using the SDK are on GitHub in the [Intersight Python Utilties repo](https://github.com/CiscoDevNet/intersight-python-utils).  The repo also contains a credentials.py module to simplify authentication across all of the example scripts.